In [ ]:
from llm_client.prompt_builder import  PromptBuilder
import pandas as pd
from config.settings import Settings
from content_retrieval import llm,doc_store, vector_store
from pipelines.processors import summarize_new_documents, add_new_documents_to_docstore
import sys
settings = Settings()


In [ ]:
kme_vertaaltabel = pd.read_csv(settings.data_root /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")

# Opbouwen Doc store van pdf

In [ ]:
kme_documents = pd.read_pickle(settings.content_folder / "extracted_df_km.pkl",compression='gzip').reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()

In [ ]:
results = add_new_documents_to_docstore(kme_documents,doc_store)

### Samenvatten en segmenteren content

In [ ]:
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')

In [ ]:
import logging
import sys

# Set up logging to see all debug messages from the Azure SDK
logger = logging.getLogger('azure')
logger.setLevel(logging.DEBUG)

# Create a handler to write logs to standard out
handler = logging.StreamHandler(stream=sys.stdout)
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [ ]:
stats = summarize_new_documents(
    doc_store=doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    max_workers=20,
    start=0,
    count=7000,
    show_progress=True
)

In [ ]:
doc_store.save()
doc_store.rebuild_search_index()

In [8]:
from dataclasses import dataclass, asdict
import json

def save(dstore):
        """Saves the current document dictionary to a Parquet file."""
        if not dstore.documents:
            # Save an empty DataFrame with the correct schema
            df = pd.DataFrame(columns=['id', 'title', 'content', 'metadata'])
        else:
            # Convert dict of Document objects to a list of dicts
            doc_list = [asdict(doc) for doc in dstore.documents.values()]
            df = pd.DataFrame.from_records(doc_list)
            
            # Serialize the metadata dict to a JSON string for Parquet compatibility
            df['metadata'] = df['metadata'].apply(json.dumps)
        persist= '/home/azureuser/cloudfiles/code/Users/a.melzer/Contentcreatie/data/docstores/kme_content/documents.parquet'
        try:
            # Save to Parquet using pyarrow engine
            df.to_parquet(persist, index=False, engine='pyarrow')
        except Exception as e:
            print(f"Error saving Parquet file {persist}: {e}")

In [9]:
save(doc_store)

In [6]:
doc_store.persistence_file

'/home/azureuser/cloudfiles/code/Users/a.melzer/Contentcreatie/data/docstores/kme_content/documents.pkl'